<a href="https://colab.research.google.com/github/knagasaki/IIIF_DL/blob/master/NDLOCR_googlecolabversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ※はじめに
このコードは、中村覚先生の「[デジタルアーカイブシステムの技術ブログ](https://nakamura196.hatenablog.com/entry/2022/04/28/115553)」で公開されているNLD OCRを利用するためのコードを、IIIF manifestを用いてまとめて処理できるように改良したものです。

# 0. GPUの情報を確認する

In [ ]:
!nvidia-smi

# 1. NDLOCRのリポジトリをcloneする(--recursiveを忘れずに！)

In [ ]:
!git clone --recursive https://github.com/ndl-lab/ndlocr_cli

# 2. 必要なパッケージをインストールする

In [ ]:
PROJECT_DIR="/content/ndlocr_cli"

In [ ]:
!pip install -r {PROJECT_DIR}/requirements.txt
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install mmcv-full==1.4.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
##numpyのバージョン問題でcolabでは動かなかったのでアップデートする(参考:https://stackoverflow.com/questions/66060487/valueerror-numpy-ndarray-size-changed-may-indicate-binary-incompatibility-exp)
!pip install --upgrade numpy

In [ ]:
%cd {PROJECT_DIR}/src/ndl_layout/mmdetection
!python setup.py bdist_wheel
!pip install dist/*.whl
%cd /content

# 4. OCRに必要な学習済みモデルをダウンロードする

In [ ]:
%cd {PROJECT_DIR}
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/ndl_layout_config.py -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P ./src/separate_pages_ssd/ssd_tools
%cd /content/

# 5. 環境変数を追加する

In [ ]:
import os
os.environ["PYTHONPATH"]=os.environ["PYTHONPATH"]+":"+f"{PROJECT_DIR}/src/text_recognition/deep-text-recognition-benchmark"

# 6. PDFを画像に変換するためのパッケージのインストール

In [ ]:
!apt-get install poppler-utils
!pip install pdf2image 

# 7. テキスト化したいIIIF化資料のJPEG画像ファイルをダウンロードする
今回は、国立国会図書館デジタルコレクションが提供している
国訳大蔵経. 第1冊（ https://dl.ndl.go.jp/info:ndljp/pid/1210630 ）をダウンロードしてみます。IIIF ManifestのURLは書誌情報の項目に記載されています。今回は以下のものになります。

https://www.dl.ndl.go.jp/api/iiif/1210630/manifest.json



In [ ]:
!mkdir -p /content/kdzk/img
import urllib.request
import json
import sys

url = 'https://www.dl.ndl.go.jp/api/iiif/1210630/manifest.json' #'http://www2.dhii.jp/nijl/NIJL0018/099-0014/manifest.json'
readObj = urllib.request.urlopen(url)
data = json.loads(readObj.read().decode('utf-8'))

arImageUrls = []
for seq in data['sequences']:
     for canvas in seq['canvases']:
         for image in canvas['images']:
             arImageUrls.append(image['resource']['service']['@id'])

tLen = len(str(len(arImageUrls))) + 1
fileNumber = 1
print ('downloading...')
for imageUrl in arImageUrls:
    localFilename = "/content/kdzk/img/"+str(fileNumber).zfill(tLen)+'.jpg'
    imageUrl = imageUrl+'/full/full/0/default.jpg'
    urllib.request.urlretrieve(imageUrl, localFilename)
    fileNumber = fileNumber + 1
    print (localFilename)

# 9. OCRの実行

/content/tygz-001以下のimgディレクトリ内の画像を処理し、
/content/tygz-001_outputに出力する場合

In [ ]:
%cd {PROJECT_DIR}
!python main.py infer /content/kdzk /content/kdzk_output -s s -x

# 10. 結果の確認

In [ ]:
import glob
import os
for fpath in sorted(glob.glob("/content/kdzk_output*/kdzk/txt/*_main.txt")):
    with open(fpath) as f:
        txtdata=f.read()
        print(os.path.basename(fpath).replace("_main.txt",""))
        print(txtdata)